In [120]:
import pandas as pd
import pickle

In [107]:
df = pd.read_pickle('/Users/zasghar.19/Desktop/eeps1720/NetworkAnalysis/disasters.pickle')

In [108]:
def county_to_state_fips(county_fips):
    return int(str(county_fips).zfill(5)[:2])

In [109]:
# Mapping to names
state_fips_to_code = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD',
    25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ',
    35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}

In [112]:
year_df = df['2006']

In [113]:
year_df['state fips'] = year_df['fips'].apply(county_to_state_fips)
year_df['state'] = year_df['state fips'].map(state_fips_to_code)

In [114]:
year_df = year_df[year_df['state'].notna()]

In [115]:
filtered_df = year_df[['ihProgramDeclared', 'iaProgramDeclared', 'paProgramDeclared', 'hmProgramDeclared', 'disasters', 'state']]

In [116]:
filtered_df = filtered_df.groupby('state')[['ihProgramDeclared', 'iaProgramDeclared', 'paProgramDeclared', 'hmProgramDeclared', 'disasters']].sum()

In [87]:
years = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']


for year in years:
    year_df = df[year]
    year_df['state fips'] = year_df['fips'].apply(county_to_state_fips)
    year_df['state'] = year_df['state fips'].map(state_fips_to_code)
    year_df = year_df[year_df['state'].notna()]
    cols = ['ihProgramDeclared', 'iaProgramDeclared', 'paProgramDeclared', 'hmProgramDeclared', 'disasters']
    filtered_df = year_df[cols + ['state']]
    year_cols = [col + year for col in cols]
    filtered_df.columns = year_cols + ['state']
    filtered_df = filtered_df[year_cols + ['state']]
    filtered_df = filtered_df.groupby('state')[year_cols].sum()
    # print(filtered_df.columns)
    if year == '1991':
        total_df = filtered_df
    else:
        total_df = pd.merge(total_df, filtered_df, how='outer', left_on=['state'], right_on=['state']).fillna(0)


In [89]:
disasters_cols = ['disasters' + year for year in years]
ih_cols = ['ihProgramDeclared' + year for year in years]
ia_cols = ['iaProgramDeclared' + year for year in years]
pa_cols = ['paProgramDeclared' + year for year in years]
hm_cols = ['hmProgramDeclared' + year for year in years]

In [99]:
total_df['disasters_median'] = total_df[disasters_cols].median(axis=1)
total_df['ihProgramDeclared_median'] = total_df[ih_cols].median(axis=1)
total_df['iaProgramDeclared_median'] = total_df[ia_cols].median(axis=1)
total_df['paProgramDeclared_median'] = total_df[pa_cols].median(axis=1)
total_df['hmProgramDeclared_median']  = total_df[hm_cols].median(axis=1)

total_df['disasters_mean'] = total_df[disasters_cols].mean(axis=1)
total_df['ihProgramDeclared_mean'] = total_df[ih_cols].mean(axis=1)
total_df['iaProgramDeclared_mean'] = total_df[ia_cols].mean(axis=1)
total_df['paProgramDeclared_mean'] = total_df[pa_cols].mean(axis=1)
total_df['hmProgramDeclared_mean']  = total_df[hm_cols].mean(axis=1)

In [105]:
median_cols = ['disasters_median', 'ihProgramDeclared_median', 'iaProgramDeclared_median', 'paProgramDeclared_median', 'hmProgramDeclared_median']
mean_cols = ['disasters_mean', 'ihProgramDeclared_mean', 'iaProgramDeclared_mean', 'paProgramDeclared_mean', 'hmProgramDeclared_mean']
mean_median_df = total_df[median_cols + mean_cols]

In [106]:
with open("./disasters_mean_median.pkl", 'wb') as f:
    pickle.dump(mean_median_df, f)

In [119]:
with open("./disasters_2006.pkl", 'wb') as f:
    pickle.dump(filtered_df, f)

In [121]:
disasters_df = pd.read_pickle('./disasters_mean_median.pkl')

In [122]:
disasters_df

,disasters_median,ihProgramDeclared_median,iaProgramDeclared_median,paProgramDeclared_median,hmProgramDeclared_median,disasters_mean,ihProgramDeclared_mean,iaProgramDeclared_mean,paProgramDeclared_mean,hmProgramDeclared_mean
state,,,,,,,,,,
AK,4.5,0.0,0.0,4.5,0.0,4.80,0.75,1.00,4.80,3.55
AL,20.5,0.0,6.0,19.0,5.0,34.15,8.15,15.40,29.75,15.50
AR,20.5,0.0,0.0,18.5,0.0,37.10,5.10,11.00,35.15,17.15
AZ,3.0,0.0,0.0,3.0,0.0,7.50,0.00,1.25,7.05,2.60
CA,24.5,0.0,10.0,14.0,0.5,30.35,1.90,17.00,27.15,7.55
CO,4.0,0.0,0.0,3.5,0.0,13.45,0.10,3.55,10.35,3.80
CT,1.5,0.0,0.0,1.0,0.0,4.05,0.60,0.65,3.70,1.00
GA,21.5,0.0,4.0,18.0,0.0,41.90,4.25,17.45,38.95,13.80
IA,27.0,0.0,0.0,26.5,0.0,45.75,10.30,22.60,39.95,22.60
